# Least-square conformal map directly applied to nurbs surfaces

A use case of least square conformal maps is the mapping of mesh (triangle-surface) to a flat plane. This is normally used for textureing meshes. Given the fact that the lest-square conformal map can produce the exact flattened surface for developable surface we try to perform the lscm method on spline-surfaces to optain a unwrapped representation of the nurbs-surface.

First a collocation method should be use to solve for conformity on nurbs surfaces. If this works for developable surfaces the extension to a weighted least-square method will be explored.

## Conformal Energy

Let a suface be represented by a Nurbs Interpolation.

Then we have already a mapping between the from the uv-system to the xyz system. We now want to find another map from uv to lm which minimize a certain criteria. min(E(u, v)
 